In [1]:
import os
import pandas as pd
import numpy as np
import numpy 
import sys
from collections import *
pd.options.mode.chained_assignment = None
from tqdm import tqdm

In [2]:
os.chdir('/Users/justi/Desktop/fall19/258/assignment2/kkbox_recommender/')
# data_path = './kkbox-music-recommendation-challenge/'
# song_path = os.path.join(data_path, 'songs.csv')
# song_info = os.path.join(data_path, 'song_extra_info.csv')
# train_path = os.path.join(data_path, 'train.csv')
song = '../songs.csv'
song_info = '../song_extra_info.csv'
train = '../train.csv'
test = '../test.csv'
member = '../members.csv'

In [3]:
ls

 Volume in drive C is Windows
 Volume Serial Number is 0A6D-C265

 Directory of C:\Users\justi\Desktop\fall19\258\assignment2\kkbox_recommender

11/24/2019  08:23 PM    <DIR>          .
11/24/2019  08:23 PM    <DIR>          ..
11/24/2019  11:28 AM                51 .gitignore
11/24/2019  03:05 PM    <DIR>          .ipynb_checkpoints
11/24/2019  01:21 PM            49,540 Assignment 2.ipynb
11/24/2019  08:23 PM            28,645 CSE 258 Task 1.ipynb
11/24/2019  01:21 PM            39,515 CSE258 Final - Task 3.ipynb
11/24/2019  11:28 AM               169 README.md
               5 File(s)        117,920 bytes
               3 Dir(s)  112,563,040,256 bytes free


### Members

In [4]:
class Members():
    def __init__(self, path):
        self.members = pd.read_csv(path)
        self.preprocess()
    
    def preprocess(self):
        '''
        1. remove wierd age
        2. replace registers -> years
        3. add using date
        4. gender to three class
        '''
        #print info
        self.info()
        
        # 1
        wierd_age = list()
        for age, count in Counter(self.members['bd']).items():
            if count < 10:
                wierd_age.append(age)
        for drop_age in wierd_age:
            self.members = self.members[self.members['bd'] != drop_age]        
        
        #2 & 3
        self.members['registration_init_time'] = pd.to_datetime(self.members['registration_init_time'], format='%Y%m%d')
        self.members['expiration_date'] = pd.to_datetime(self.members['expiration_date'], format='%Y%m%d')
        self.members = self.members[self.members['expiration_date'] != '1970-01-01 00:00:00']
        self.members['activation_days'] = self.members['expiration_date'] - self.members['registration_init_time']
#         self.members['activation_days'].astype(int)
        days = self.members['activation_days'].values
        days = days.astype('timedelta64[D]')
        days = days / np.timedelta64(1, 'D')
        self.members['activation_days_int'] = days
        self.members['expiration_date'] = pd.DatetimeIndex(self.members['expiration_date']).year
        self.members['expiration_date'] -= min(self.members['expiration_date'])
        self.members['registration_init_time'] = pd.DatetimeIndex(self.members['registration_init_time']).year
        self.members['registration_init_time'] -= min(self.members['registration_init_time'])


        #4    
        self.members['gender'][self.members['gender'].isnull()] = 0
        self.members['gender'][self.members['gender'] == 'male'] = 1
        self.members['gender'][self.members['gender'] == 'female'] = -1
        
        #print info
        self.info()


    
    def info(self):
        print("The number of total users is: {}".format(len(self.members)))

In [5]:
members = Members(member)

The number of total users is: 34403
The number of total users is: 34288


In [6]:
members.members.head()

,msno,city,bd,gender,registered_via,registration_init_time,expiration_date,activation_days,activation_days_int
0,XQxgAYj3klVKjR3oxPPXYYFp4soD4TuBghkhMTD4oTw=,1,0,0,7,7,13,2223 days,2223.0
1,UizsfmJb9mV54qE9hCYyU07Va97c0lCRLEQX3ae+ztM=,1,0,0,7,11,13,725 days,725.0
2,D8nEhsIOBSoE6VthTaqDX8U6lqjJ7dLdr72mOyLya2A=,1,0,0,4,12,13,457 days,457.0
3,mCuD+tZ1hERA/o5GPqk38e041J8ZsBaLcu7nGoIIvhI=,1,0,0,9,11,11,1 days,1.0
4,q4HRBfVSssAFS9iRfxWrohxuk9kCYMKjHOEagUMV6rQ=,1,0,0,4,13,13,138 days,138.0


### Songs

In [7]:
class clean_song_data():
    def __init__(self, song, song_info):
        self.song = pd.read_csv(song)
        self.song_info = pd.read_csv(song_info)
        self.result = pd.merge(self.song, self.song_info, on='song_id')
        self.filter_1()
        self.filter_2()
        self.add_count()
        self.convert_type()
        self.convert_frequency()
        
    def filter_1(self):
        '''
        1. Filter out NaN
        2. Use only English songs
        
        '''
        header = []
        self.result = self.result.dropna()
#         for i in self.result.columns:
#             header.append(i)
#         for i in header:
#             self.result = self.result[pd.notnull(self.result[i])]
        
        # 52.0 = English
        self.result = self.result[self.result.language == 52.0]
    
    def filter_2(self):
        '''
        genre and artist count > 15
        
        '''
        header = ['genre_ids','artist_name']
        for i in header:
            k = pd.DataFrame(self.result.groupby(i).size().items())
            k = k.rename(columns={0: 'item', 1: 'count'})
            k = k[k['count']>15]
            self.result = self.result[self.result[i].isin(k['item'])]
            self.result.head()
            print(i,len(self.result))
            
    def add_count(self):
        '''
        add count for genre, composer 
        
        '''
        header = ['genre_ids', 'composer']
        name = ['gener_count', 'composer_count']
        for num,i in enumerate(header):
            l = []
            for j in self.result[i]:
                l.append(len(j.split('|')))
            self.result[name[num]] = l
            
    def convert_type(self):
        '''
        Convert str to list
        '''
        tmp = []
        for i in self.result['genre_ids'].tolist():
            tmp.append(list(map(int,i.split('|'))))
        self.result['genre_ids_list'] = tmp
    
    def convert_frequency(self):
        '''
        Convert composer, lyricist frequency
        '''
        header = ['artist_name', 'composer', 'lyricist']
        for g in header:
            col = defaultdict(int)
            for i in self.result[g].tolist():
                tmp = list(map(str,i.split('|')))
                for i in tmp:
                    col[i] += 1

            col_name_count = []
            for i in self.result[g].tolist():
                k = []
                tmp = list(map(str,i.split('|')))
                for j in range(len(tmp)):
                    k.append(col[tmp[j]])
                k = sum(k)
                col_name_count.append(k)
            self.result[g+'_count'] = col_name_count


In [8]:
song = clean_song_data(song, song_info)

genre_ids 168873
artist_name 121316


### Train

In [14]:
class Data():
    def __init__(self, path1, songs, users):
        self.train = pd.read_csv(path1)
        self.songs = songs
        self.users = users
        self.train = self.filtered(self.train)
        Ntrain = int(len(self.train)*0.8)
        Nval = int(len(self.train)*0.1)
        
        train = self.train[:Ntrain]
        val = self.train[Ntrain:Ntrain+Nval]
        test = self.train[Ntrain+Nval:]
        self.train = train
        self.val = val
        self.test = test
        
        self.people2songs = self.train.groupby('msno')["song_id"].apply(list)
        self.songs2count = self.train.groupby('song_id').size()
        self.song2user = self.train.groupby("song_id")["msno"].apply(list)
#         self.songs_set = set(self.songs['song_id'])
        self.all = pd.concat([self.train, self.test], axis=0, sort=False)
        self.people2songs_all = self.all.groupby('msno')["song_id"].apply(set)
    
    def filtered(self, pd):
        valid_user = set(self.users.members['msno'])
        valid_song = set(self.songs.result['song_id'])
        pd = pd[pd['msno'].isin(valid_user)]
        pd = pd[pd['song_id'].isin(valid_song)]
        return pd
        
# def sample(positive_data, all_songs, people2songs):
#     negative_dict = dict()
#     for people, count in tqdm(positive_data.groupby('msno').size().items()):
#         negative_dict[people] = random.sample(all_songs - people2songs[people], count)
#     append_list = []
#     for people, songs in negative_dict.items():
#         for song in songs:
#             append_list.append([people, song, 0])
#     positive_matrix = np.hstack((positive_data.values[:, :2], np.ones((positive_data.shape[0], 1))))
#     return np.vstack((positive_matrix, np.array(append_list)))

In [15]:
data = Data(train, song, members)

In [93]:
class Pop():
    def __init__(self, train, percentile=0.5):
        self.train = train
        self.pop = self.gen_pop(percentile)
    
    def gen_pop(self, percentile):
        counts = 0
        out_set = set()
        train_pos = []
        train_label = self.train['target'].values
        train_songid = self.train['song_id'].values
        for i in range(len(train_label)):
            if train_label[i] == 1:
                train_pos.append(train_songid[i])
        count = Counter(train_pos)
        s = [[j,i] for i,j in count.items()]
        s.sort(key=lambda tup: tup[0], reverse = True)
#         vc= self.train['song_id'].value_counts()
        for count, song in s:
            out_set.add(song)
            counts += count
            if counts > len(data.train)*percentile:
                break
        return out_set
    
    def baseline(self, u, song):
        if song in self.pop:
            return 1
        else:
            return 0
        
    
class Evaluation():
    def __init__(self, data):
        self.test = data.test
        
    def accuracy(self, model):
        test_matrix = self.test.values
        label = []
        pred = []
        for i in range(test_matrix.shape[0]):
            u, s, l = test_matrix[i][0], test_matrix[i][1], test_matrix[i][5]
            pred.append(model(u, s))
            label.append(l)
        
        return sum(np.array(pred) == np.array(label))/len(label)

In [94]:
percentile = 0.5
model = Pop(data.train, percentile)
eval = Evaluation(data)
print("Accuracy of prediction is {}".format(eval.accuracy(model.baseline)))

Accuracy of prediction is 0.4796674834827383


### Jaccard

In [142]:
class Jac():
    def __init__(self, data):
        self.data = data
        self.user2song = defaultdict(set)
        self.song2user = defaultdict(set)
        self.create()
        
    def create(self):
        self.train_pair = self.data.train[['msno','song_id']].values
        self.val_pair = self.data.val[['msno','song_id']].values
        self.test_pair = self.data.test[['msno','song_id']].values
        for user, song in self.train_pair:
            self.user2song[user].add(song)
            self.song2user[song].add(user)
        
    def Jaccard(self, s1, s2):
        numer = len(s1.intersection(s2))
        denom = len(s1.union(s2))
        return numer / denom

    def cal_acc(self, tmp):
        test_ground_truth = self.data.test['target'].values
        count = 0
        for num,i in enumerate(test_ground_truth):
            if test_ground_truth[num] == tmp[num]:
                count += 1
        acc = count/len(tmp)
        print('Jaccard accuracy on test set:{}'.format(acc))
        return acc

    def use_jac(self, thres):
        tmp = []
        count = 0
        for user, song in self.test_pair:
            count += 1
            for s in self.user2song[user]:
                score = self.Jaccard(self.song2user[song], self.song2user[s])
                if score > thres:
                    tmp.append(1)
                    break
            if len(tmp) != count:
                tmp.append(0)

        acc = self.cal_acc(tmp)
        return acc, tmp
    
    def use_jac_avg(self, thres):
        out = []
        count = 0
        for user, song in self.test_pair:
            tmp = []
            count += 1
            for s in self.user2song[user]:
                score = self.Jaccard(self.song2user[song], self.song2user[s])
                tmp.append(score)
            tmp.sort(reverse = True)
            if len(tmp[:3]) == 0:
                out.append(0)
                continue
            avg = sum(tmp[:3])/len(tmp[:3])
            if avg > thres:
                out.append(1)
                continue
            if len(out) != count:
                out.append(0)
        acc = self.cal_acc(out)
        return acc
        

#     def use_jac_feature(self):
#         out1, out2 = [], []
#         for user, song in self.train_pair:
#             tmp = []
#             for u in self.song2user[song]:
#                 score = self.Jaccard(self.user2song[user], self.user2song[u])
#                 tmp.append(score)
#             if len(tmp[:3]) == 0:
#                 out1.append(0)
#                 continue
#             avg = sum(tmp[:3])/len(tmp[:3])
#             out1.append(avg)
#         for user, song in self.train_pair:
#             tmp = []
#             for s in self.user2song[user]:
#                 score = self.Jaccard(self.song2user[song], self.song2user[s])
#                 tmp.append(score)
#             if len(tmp[:3]) == 0:
#                 out2.append(0)
#                 continue
#             avg = sum(tmp[:3])/len(tmp[:3])
#             out2.append(avg)       
#         return out1, out2
    
    def feature_song(self, user, song):
        tmp = []
        for u in self.song2user[song]:
            score = self.Jaccard(self.user2song[user], self.user2song[u])
            tmp.append(score)
        if len(tmp[:3]) == 0:
            top = 0
        else:
            top = sum(tmp[:3])/len(tmp[:3])
            
        if len(tmp[-3:]) == 0:
            tail = 0
        else:
            tail = sum(tmp[-3:])/len(tmp[-3:])
            
        if len(tmp[:]) == 0:
            avg = 0
        else:
            avg = sum(tmp[:])/len(tmp[:])
        return top,avg,tail
    
    def feature_user(self, user, song):
        tmp = []
        for s in self.user2song[user]:
            score = self.Jaccard(self.song2user[song], self.song2user[s])
            tmp.append(score)
        if len(tmp[:3]) == 0:
            top = 0
        else:
            top = sum(tmp[:3])/len(tmp[:3])
            
        if len(tmp[-3:]) == 0:
            tail = 0
        else:
            tail = sum(tmp[-3:])/len(tmp[-3:])
            
        if len(tmp[:]) == 0:
            avg = 0
        else:
            avg = sum(tmp[:])/len(tmp[:])
        return top,avg,tail
                

In [143]:
jac = Jac(data)

In [144]:
thres = 0.5
# acc, pred = jac.use_jac(thres)
# acc = jac.use_jac_avg(thres)
out1, out2 = jac.use_jac_feature() # top 3

### Popularity & Jaccard

In [95]:
percentile = 0.2
model = Pop(data.train, percentile)
top_song = model.gen_pop(percentile)

In [97]:
pop_test = []
test_pair = data.test[['msno','song_id']].values
for user, song in test_pair:
    if song in top_song:
        pop_test.append(1)
    else:
        pop_test.append(0)

In [98]:
comb_test = []
for i in range(len(pop_test)):
    if pop_test[i] == 1 and pred[i] == 1:
        comb_test.append(1)
    else:
        comb_test.append(0)

In [101]:
def cal_acc(test, tmp):
    test_ground_truth = test['target'].values
    count = 0
    for num,i in enumerate(test_ground_truth):
        if test_ground_truth[num] == tmp[num]:
            count += 1
    acc = count/len(tmp)
    print('Jaccard accuracy on test set:{}'.format(acc))
    return acc

acc = cal_acc(data.test, comb_test)

Jaccard accuracy on test set:0.5849747200595552
